In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('raw_data_v2.csv')

print(df.columns)

Index(['Unnamed: 0', 'title', 'price', 'n_rooms', 'area', 'year',
       'price_per_sqft', 'size_sqft', 'road_name', 'n_bedrooms', 'n_bathrooms',
       'region', 'scraped_date'],
      dtype='object')


In [71]:
df

,Unnamed: 0,title,price,n_rooms,area,year,price_per_sqft,size_sqft,road_name,n_bedrooms,n_bathrooms,region,scraped_date
0,0,111A DEPOT ROAD,"S$ 970,000",HDB 5-Room,Bukit Merah,2004,S$ 784 psf,"1,238 sqft",Depot Road,3 beds,2 baths,central,2025-03-27
1,1,405 SIN MING AVENUE,"S$ 1,050,000",HDB 5-Room,Bishan,1985,S$ 813 psf,"1,292 sqft",Sin Ming Avenue,3 beds,2 baths,central,2025-03-27
2,2,61C STRATHMORE AVENUE,"S$ 820,000",HDB 3-Room,Queenstown,2010,"S$ 1,104 psf",743 sqft,Strathmore Avenue,2 beds,2 baths,central,2025-03-27
3,3,87 ZION ROAD,"S$ 1,158,000",HDB 5-Room,Bukit Merah,1974,S$ 944 psf,"1,227 sqft",Zion Road,3 beds,2 baths,central,2025-03-27
4,4,50 DORSET ROAD,"S$ 615,000",HDB 3-Room,Kallang/Whampoa,1978,S$ 697 psf,882 sqft,Dorset Road,2 beds,2 baths,central,2025-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,992,180A RIVERVALE CRESCENT,"S$ 530,000",HDB 3-Room,Sengkang,2012,S$ 735 psf,721 sqft,Rivervale Crescent,2 beds,2 baths,north-east,2025-03-19
993,993,439A SENGKANG WEST AVENUE,"S$ 708,000",HDB 4-Room,Sengkang,2009,S$ 700 psf,"1,011 sqft",Sengkang West Avenue,3 beds,2 baths,north-east,2025-03-19
994,994,173 HOUGANG AVENUE 1,"S$ 628,000",HDB 4-Room,Hougang,1996,S$ 572 psf,"1,098 sqft",Hougang Avenue 1,3 beds,2 baths,north-east,2025-03-19
995,995,266B COMPASSVALE BOW,"S$ 710,000",HDB 4-Room,19,99 years,S$ 717 psf,990 sqft,Compassvale Bow,3 beds,2 baths,north-east,2025-03-19


In [73]:
#Common case 1: values are in the wrong columns, such as road name in sqft column, size sqft in psf column -> move all to the right by 1 step (column)

def is_price_per_sqft(val):
    return isinstance(val, str) and 'psf' in val

def is_size_sqft(val):
    return isinstance(val, str) and 'sqft' in val

def is_road_name(val):
    return isinstance(val, str) and any(word in val.lower() for word in ['street', 'road', 'avenue', 'drive', 'boulevard', 'lorong'])

def is_year(val):
    return isinstance(val,str) and val.isdigit() and len(val)==4

for idx, row in df.iterrows():
    
    if is_price_per_sqft(row['year']): # check if price_per_sqft is mistakenly in year column
        df.at[idx, 'road_name'] = row['size_sqft'] #update road_name column as the val in size_sqft column
        df.at[idx, 'size_sqft'] = row['price_per_sqft']
        df.at[idx, 'price_per_sqft'] = row['year']
        if is_year(row['area']): # check if year is mistakenly in area column
            df.at[idx, 'year'] = row['area']  
        df.at[idx, 'year'] = "" # or fillna if needed

In [74]:
#Common case 2: area replaced with district code (numbers) -> replace with region

for idx, row in df.iterrows():
    if row['area'].isdigit():
        df.at[idx, 'area'] = row['region'].capitalize()
        

In [75]:
#Common case 3: year column is either empty string/non-year strings -> replace with median year of properties from the region

all_regions = set(df['region'])
for r in all_regions:
    df_a = df[df['region'] == r].copy()

    # Get the median year, ignoring non-numeric values
    median_year = pd.to_numeric(df_a['year'], errors='coerce').dropna().median()

    # Replace all non-digit year values with the median year
    df_a['year'] = df_a['year'].apply(
        lambda x: str(median_year) if not str(x).isdigit() else x
    )

    # Write the cleaned column back to the original dataframe
    df.loc[df['region'] == r, 'year'] = df_a['year']

print('no. of null rows of year:', len(df[df['year'] == ""]))

no. of null rows of year: 0


In [76]:
df 

,Unnamed: 0,title,price,n_rooms,area,year,price_per_sqft,size_sqft,road_name,n_bedrooms,n_bathrooms,region,scraped_date
0,0,111A DEPOT ROAD,"S$ 970,000",HDB 5-Room,Bukit Merah,2004,S$ 784 psf,"1,238 sqft",Depot Road,3 beds,2 baths,central,2025-03-27
1,1,405 SIN MING AVENUE,"S$ 1,050,000",HDB 5-Room,Bishan,1985,S$ 813 psf,"1,292 sqft",Sin Ming Avenue,3 beds,2 baths,central,2025-03-27
2,2,61C STRATHMORE AVENUE,"S$ 820,000",HDB 3-Room,Queenstown,2010,"S$ 1,104 psf",743 sqft,Strathmore Avenue,2 beds,2 baths,central,2025-03-27
3,3,87 ZION ROAD,"S$ 1,158,000",HDB 5-Room,Bukit Merah,1974,S$ 944 psf,"1,227 sqft",Zion Road,3 beds,2 baths,central,2025-03-27
4,4,50 DORSET ROAD,"S$ 615,000",HDB 3-Room,Kallang/Whampoa,1978,S$ 697 psf,882 sqft,Dorset Road,2 beds,2 baths,central,2025-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...
992,992,180A RIVERVALE CRESCENT,"S$ 530,000",HDB 3-Room,Sengkang,2012,S$ 735 psf,721 sqft,Rivervale Crescent,2 beds,2 baths,north-east,2025-03-19
993,993,439A SENGKANG WEST AVENUE,"S$ 708,000",HDB 4-Room,Sengkang,2009,S$ 700 psf,"1,011 sqft",Sengkang West Avenue,3 beds,2 baths,north-east,2025-03-19
994,994,173 HOUGANG AVENUE 1,"S$ 628,000",HDB 4-Room,Hougang,1996,S$ 572 psf,"1,098 sqft",Hougang Avenue 1,3 beds,2 baths,north-east,2025-03-19
995,995,266B COMPASSVALE BOW,"S$ 710,000",HDB 4-Room,North-east,2010.0,S$ 717 psf,990 sqft,Compassvale Bow,3 beds,2 baths,north-east,2025-03-19


In [77]:
df.to_csv('cleaned_data_v2.csv') 

In [ ]:
# length and null checks on cleaned df 
print(len(df[df['price_per_sqft'].str.contains('psf')]))
print(len(df[df['size_sqft'].str.contains('sqft')]))
print(len(df[df['n_bathrooms'].str.contains('bath')]))
print(len(df[df['n_bedrooms'].str.contains('bed')]))
print(len(df[df['title'] != ""]))
print('no. of null rows of year:', len(df[df['year'] == ""]))
print('no. of null rows of road:', len(df[df['road_name'] == ""]))


997
997
997
997
997
no. of null rows of year: 0
no. of null rows of road: 0
